In [ ]:
import transformers

device = 'cuda:1'
tokenizer = transformers.AutoTokenizer.from_pretrained('EleutherAI/gpt-j-6B')
tokenizer.pad_token = tokenizer.eos_token
model = transformers.AutoModelForCausalLM.from_pretrained('EleutherAI/gpt-j-6B')
model.to(device)

In [ ]:
model.config

In [ ]:
import json
with open('../data/occupations.json', 'r') as handle:
    entries = json.load(handle)
occupations = {entry['occupation'] for entry in entries}

In [ ]:
import random
choice = random.choice(entries)
print([entry for entry in entries if entry['entity'] == choice['entity']])

In [ ]:
import torch
entity = 'Jimmy Carter'
# prompt = 'Everybody knows {entity} is a {occupation}'
prompt = '{entity}, the {occupation}'
sentences = [
    prompt.format(entity=entity, occupation=occupation)
    for occupation in occupations
]
inputs = tokenizer(sentences, return_tensors='pt', padding='longest').to(device)
with torch.inference_mode():
    outputs = model(**inputs, output_hidden_states=True, return_dict=True)
logits = outputs.logits

In [ ]:
scores = []
for ids, ls in zip(inputs.input_ids, logits):
    score = 0
    logprobs = torch.log_softmax(ls, dim=-1)
    for position, token_id in enumerate(ids[1:]):
        if token_id.item() in {tokenizer.eos_token_id, tokenizer.pad_token_id}:
            continue
        score += logprobs[position, token_id]
    scores.append(score.item())
topk = torch.tensor(scores).topk(k=100)

for position, index in enumerate(topk.indices.tolist()):
    print(sentences[index], f'{topk.values[position].item():.3f}')